**Required Libraries**

In [ ]:
from collections import defaultdict

import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from skimage.exposure import is_low_contrast

**Constant definitions**

In [ ]:
DATA_DIR = "./data/images"
ANNOT_DIR = "./data/annotations"

**Utility Functions**

In [ ]:
def clamp(value, minimum, maximum):
    return min(maximum, max(minimum, value))

In [ ]:
def display_bgr_image(bgr_image, show=True):
    rgb_image = cv.cvtColor(bgr_image, cv.COLOR_BGR2RGB)
    fig, ax = plt.subplots()
    ax.imshow(rgb_image)
    ax.axis(False)
    if show:
        plt.show()
    return fig, ax

def display_gray_image(gray_image, show=True):
    fig, ax = plt.subplots()
    ax.imshow(gray_image, cmap="gray")
    ax.axis(False)
    if show:
        plt.show()
    return fig, ax

In [ ]:
def weighted_gray_transform(bgr_image, weights=[0.114, 0.587, 0.299]):
    m = np.array(weights).reshape((1,3))
    return cv.transform(bgr_image, m)

# Original Image

In [ ]:
selected_image = 873
src_image = cv.imread(f"{DATA_DIR}/road{selected_image}.png")

fig, ax = display_bgr_image(src_image)
plt.clf();

# Image Processing
In this phase, the image will be treated in an attempt to reduce possible noise from environment and/or weather, this will be achieved by applying **CLAHE equalization** on the image, followed by an **automatic brightness and contrast correction** and finishing with the initial step of **meanShift**, this is the filtering stage of the **meanshift** segmentation that flattens color gradients and fine-grain textures of the image (see `pyrMeanShiftFiltering`).

## CLAHE Equalization

In [ ]:
def hsv_clahe_equalization(bgr_image, clipLimit = 2.0, tileGridSize = (8, 8)):
    hsv_image = cv.cvtColor(bgr_image, cv.COLOR_BGR2HSV)

    h, s, v = cv.split(hsv_image)

    clahe = cv.createCLAHE(
        clipLimit=clipLimit,
        tileGridSize=tileGridSize
    )

    s_equalized = clahe.apply(s)
    v_equalized = clahe.apply(v)

    equalized_image = cv.merge([h, s_equalized, v_equalized])
    return cv.cvtColor(equalized_image, cv.COLOR_HSV2BGR)

clahe_image = hsv_clahe_equalization(src_image, clipLimit=2.0, tileGridSize=(8, 8))
fig, ax = display_bgr_image(clahe_image)
plt.clf();

## Automatic Brightness and Contrast Correction

In [ ]:
def automatic_brightness_contrast(bgr_image, clip_hist_percent = 0.01, use_scale_abs = True, return_verbose = False):
    gray_image = cv.cvtColor(bgr_image, cv.COLOR_BGR2GRAY)

    # Grayscale histogram of the image
    hist = cv.calcHist([gray_image], [0], None, [256], [0, 256])
    hist_size = len(hist)

    # Cumulative distribution of the histogram
    acc = []
    acc.append( float(hist[0]) )
    for i in range(1, hist_size):
        acc.append( acc[i - 1] + float(hist[i]) )
    
    # Locate points to clip
    maximum = acc[-1]
    clip_hist = clip_hist_percent * maximum / 2.0

    # Left cut
    minimum_gray = 0
    while acc[minimum_gray] < clip_hist:
        minimum_gray += 1

    # Right cut
    maximum_gray = hist_size - 1
    while acc[maximum_gray] >= (maximum - clip_hist):
        maximum_gray -= 1

    # Calculate alpha and beta values for the scaling
    alpha = 255 / (maximum_gray - minimum_gray)
    beta = - minimum_gray * alpha

    if use_scale_abs:
        processed_image = cv.convertScaleAbs(bgr_image, alpha=alpha, beta=beta)
    else:
        processed_image = bgr_image * alpha + beta
        processed_image[processed_image < 0] = 0
        processed_image[processed_image > 255] = 255

    if return_verbose:
        processed_hist = cv.calcHist([gray_image], [0], None, [256], [minimum_gray, maximum_gray])

        return processed_image, alpha, beta, hist, processed_hist
    
    return processed_image

contrast_image = automatic_brightness_contrast(clahe_image, clip_hist_percent=0.01, use_scale_abs=True)
fig, ax = display_bgr_image(contrast_image)
plt.clf();


## Mean Shift Filtering

In [ ]:
processed_image = cv.pyrMeanShiftFiltering(contrast_image, 10, 25, 100)
fig, ax = display_bgr_image(processed_image)
plt.clf();

# Color Segmentation

In this phase, the processed image will be segmented by color. It was decided to segment the reds and blues separately. Afterwards, the segmented image is binarized by application of thresholding techniques, in this case, it was decided to use Otsu technique for thresholding. Further more, it is done a small post-processing to remove small components of the image that aren't likely to be traffic signs.

## Red Segmentation

In [ ]:
def segment_reds(bgr_image):
    smooth_image = cv.edgePreservingFilter(bgr_image, flags=cv.NORMCONV_FILTER, sigma_s=10, sigma_r=0.2)
    hsv_image = cv.cvtColor(smooth_image, cv.COLOR_BGR2HSV)

    # Red zones
    lowerbound_1 = np.array([0, 40, 40])
    upperbound_1 = np.array([15, 255, 255])

    lowerbound_2 = np.array([135, 40, 40])
    upperbound_2 = np.array([180, 255, 255])

    red_1 = cv.inRange(hsv_image, lowerbound_1, upperbound_1)
    red_2 = cv.inRange(hsv_image, lowerbound_2, upperbound_2)
    mask = cv.bitwise_or(red_1, red_2)


    segmented_image = cv.bitwise_and(smooth_image, smooth_image, mask=mask)
    BIN_THRESHOLD = 0.75

    gray_image = weighted_gray_transform(segmented_image, [0, 0, 1])

    threshold = int(255 * BIN_THRESHOLD)
    ret_thresh, gray_image = cv.threshold(gray_image, threshold, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)

    return gray_image

red_segmented_image = segment_reds(src_image)
fig, ax = display_gray_image(red_segmented_image)
plt.clf();

## Blue Segmentation

In [ ]:
def segment_blues(bgr_image):
    smooth_image = cv.edgePreservingFilter(bgr_image, flags=cv.NORMCONV_FILTER, sigma_s=50, sigma_r=0.5)
    hsv_image = cv.cvtColor(smooth_image, cv.COLOR_BGR2HSV)

    # Blue zones
    lowerbound = np.array([100, 40, 70])
    upperbound = np.array([140, 255, 255])

    mask = cv.inRange(hsv_image, lowerbound, upperbound)
    
    segmented_image = cv.bitwise_and(smooth_image, smooth_image, mask=mask)
    BIN_THRESHOLD = 0.25

    # gray_image = cv.cvtColor(segmented_image, cv.COLOR_BGR2GRAY)
    gray_image = weighted_gray_transform(segmented_image, [1, 0, 0])

    threshold = int(255 * BIN_THRESHOLD)
    ret_thresh, gray_image = cv.threshold(gray_image, threshold, 255, cv.THRESH_BINARY)
    # black_img = np.zeros(shape=bgr_image.shape[0:2], dtype=np.uint8)
    # white_img = np.ones(shape=bgr_image.shape[0:2], dtype=np.uint8)*255
    
    # gray_image = cv.bitwise_or(black_img, white_img, mask=mask)

    return gray_image

blue_segmented_image = segment_blues(processed_image)
fig, ax = display_gray_image(blue_segmented_image)
plt.clf();

## Post-segmentation Cleanup

TODO: rethink the `removeSmallComponents`

# RoI Extraction

In this phase, it will be extracted the regions of interest (RoI) of the segmented images, those will be considered the potential traffic signs for the next phase. To do this extraction, it will be applied an edge detector followed by extracting the contours and then the bounding rectangle of that contour as the RoI.

## Edge Detection

In [ ]:
def edge_detection(gray_image):
    # Apply morphological operation to soften possible artefacts
    kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, ksize=(3, 3))
    morph_image = cv.morphologyEx(gray_image, cv.MORPH_CLOSE, kernel, iterations=1)

    return cv.Canny(morph_image, threshold1=200, threshold2=240, apertureSize=5)

    

red_edges = edge_detection(red_segmented_image)
print("Edges of Red Segmented Image")
fig, ax = display_gray_image(red_edges)
plt.clf();

blue_edges = edge_detection(blue_segmented_image)
print("Edges of Blue Segmented Image")
fig, ax = display_gray_image(blue_edges)
plt.clf();


## Extract RoI

Regions of interest will be bounding rectangles of the contours detected in the image processed by Canny edge detector. To eliminate redundant RoI and false positives, RoI that are contained completely inside another will be merged, and RoI which don't meet certain criteria will also be discarded.

In [ ]:
def mergeROI(rois):
    # Sort RoI by X-coordinate and width to merge RoI
    sorted_rois = sorted(rois, key=lambda roi: (roi[0][0], -roi[0][2], roi[0][1], -roi[0][3]))

    i = 0
    while True:
        if i >= len(sorted_rois):
            break

        pivot = sorted_rois[i]
        pivot_x0, pivot_y0, pivot_x1, pivot_y1 = pivot[0][0], pivot[0][1], pivot[0][0] + pivot[0][2], pivot[0][1] + pivot[0][3]
        j = i + 1
        while True:
            if j >= len(sorted_rois):
                break

            other = sorted_rois[j]
            other_x0, other_y0, other_x1, other_y1 = other[0][0], other[0][1], other[0][0] + other[0][2], other[0][1] + other[0][3]
        
            # Beginning of other RoI is already past the ending of the pivot RoI
            if other_x0 > pivot_x1:
                break

            # Check if it's inside, and delete if so, otherwise advance
            if other_y0 > pivot_y0 and other_y1 < pivot_y1 and other_x1 < pivot_x1:
                sorted_rois.pop(j)
            else:
                j += 1
        
        i += 1
    return sorted_rois


def extractROI(edge_image, red_image, blue_image, roi_type):
    kernel = cv.getStructuringElement(shape=cv.MORPH_ELLIPSE, ksize=(3, 3))
    morph_image = cv.morphologyEx(edge_image, cv.MORPH_DILATE, kernel, iterations=1)

    contours, hierarchy = cv.findContours(morph_image, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)

    # Apply convex hulls to close off shapes
    contours = [cv.convexHull(contour) for contour in contours]

    # Approximate contour
    contours = [cv.approxPolyDP(contour, 0.01 * cv.arcLength(contour, True), True) for contour in contours]

    rois = [(cv.boundingRect(contour), contour) for contour in contours]

    rois = mergeROI(rois)

    i = 0
    while True:
        if i >= len(rois):
            break

        (x, y, w, h), contours = rois[i]
        roi_size = w * h

        SIZE_THRESHOLD = 15 * 15
        if roi_size < SIZE_THRESHOLD:
            rois.pop(i)
            continue

        ASPECT_RATIO_MIN = 0.45
        ASPECT_RATIO_MAX = 1.55
        aspect_ratio = float(w) / h
        if aspect_ratio < ASPECT_RATIO_MIN or aspect_ratio > ASPECT_RATIO_MAX: 
            rois.pop(i)
            continue

        blue_pixels = 0
        red_pixels = 0
        for xi in range(x, x+w):
            for yi in range(y, y+h):
                if cv.pointPolygonTest(contours, (xi, yi), measureDist=False) >= 0:
                    if red_image[yi, xi] > 127:
                        red_pixels += 1
                    if blue_image[yi, xi] > 127:
                        blue_pixels += 1

        blue_ratio = blue_pixels / roi_size
        red_ratio = red_pixels / roi_size
        red_blue_ratio = red_pixels / (blue_pixels + 1e-4)

        if roi_type == "red":
            if red_ratio < 0.10:
                rois.pop(i)
                continue
        elif roi_type == "blue":
            if blue_ratio < 0.40:
                rois.pop(i)
                continue
        
        rois[i] = ((x, y, w, h), contours, red_ratio, blue_ratio, red_blue_ratio)

        i += 1
    
    return rois

red_roi = extractROI(red_edges, red_image=red_segmented_image, blue_image=blue_segmented_image, roi_type="red")

blue_roi = extractROI(blue_edges, red_image=red_segmented_image, blue_image=blue_segmented_image, roi_type="blue")

roi_red_image = np.zeros(shape=(red_edges.shape + (3,)), dtype=np.uint8)
print("Red Regions of Interest")
for roi in red_roi:
    (x, y, w, h), contours, red_ratio, blue_ratio, red_blue_ratio = roi
    cv.rectangle(roi_red_image, (int(x), int(y)), (int(x+w), int(y+h)), (197, 183, 255), 1)
    cv.drawContours(roi_red_image, [contours], 0, color=(255, 255, 255))
    region = np.zeros_like(red_edges[y:y+h, x:x+w])
    cv.drawContours(region, [contours], 0, color=(255), offset=(-x, -y))
    fig, ax = display_gray_image(region)
    plt.clf();

display_bgr_image(roi_red_image)
plt.clf();

print("Blue Regions of Interest")
roi_blue_image = np.zeros(shape=(red_edges.shape + (3,)), dtype=np.uint8)
for roi in blue_roi:
    (x, y, w, h), contours, red_ratio, blue_ratio, red_blue_ratio = roi
    cv.rectangle(roi_blue_image, (int(x), int(y)), (int(x+w), int(y+h)), (197, 183, 255), 1)
    cv.drawContours(roi_blue_image, [contours], 0, color=(255, 255, 255))
    region = np.zeros_like(blue_edges[y:y+h, x:x+w])
    # for x0 in range(x, x+w):
    #     for y0 in range(y, y+h):
    #         if cv.pointPolygonTest(contours, (x0, y0), measureDist=False) >= 0:
    #             region[y0-y, x0-x] = 255
    cv.drawContours(region, [contours], 0, color=(255), offset=(-x, -y))
    fig, ax = display_gray_image(region)
    plt.clf();

display_bgr_image(roi_blue_image)
plt.clf();

# Shape Detection

## Corner Detection

In [ ]:
def corner_detection(roi, gray_image):
    (x, y, w, h), contours, _, _, _ = roi
    region = np.zeros(shape=(h, w), dtype=np.uint8)
    cv.drawContours(region, [contours], 0, color=(255), offset=(-x, -y))
    region_32f = np.float32(region)

    corners = cv.cornerHarris(region_32f, 6, 3, 0.04)
    corners = cv.dilate(corners, None)

    norm_corners = np.empty(corners.shape, dtype=np.float32)
    cv.normalize(corners, norm_corners, 255.0, 0.0, cv.NORM_INF)
    norm_corners = cv.convertScaleAbs(norm_corners)

    CORNER_THRESHOLD = 60

    w_20, h_20 = int(0.20 * w), int(0.20 * h)
    tl = 0.25 * (norm_corners[0:h_20, 0:w_20].max() > CORNER_THRESHOLD)
    tc = 0.25 * (norm_corners[0:h_20, 2*w_20:3*w_20].max() > CORNER_THRESHOLD)
    tr = 0.25 * (norm_corners[0:h_20, 4*w_20:5*w_20].max() > CORNER_THRESHOLD)

    ml = 0.25 * (norm_corners[2*h_20:3*h_20, 0:w_20].max() > CORNER_THRESHOLD)
    mr = 0.25 * (norm_corners[2*h_20:3*h_20, 4*w_20:5*w_20].max() > CORNER_THRESHOLD)

    bl = 0.25 * (norm_corners[4*h_20:5*h_20, 0:w_20].max() > CORNER_THRESHOLD)
    bc = 0.25 * (norm_corners[4*h_20:5*h_20, 2*w_20:3*w_20].max() > CORNER_THRESHOLD)
    br = 0.25 * (norm_corners[4*h_20:5*h_20, 4*w_20:5*w_20].max() > CORNER_THRESHOLD)

    sqp = clamp(tl + tr + br + bl - ml - mr - tc - bc, 0.0, 1.0)
    tup = clamp(1/0.75 * (bl + br + tc) - 1.1 * (tl + tr) - 0.9 * (ml + mr), 0.0, 1.0)
    tdp = clamp(1/0.75 * (tl + tr + bc) - 1.1 * (bl + br) - 0.9 * (ml + mr), 0.0, 1.0)
    circle = clamp((2 * (tc + bc + ml + mr) + 0.5 * (tl + tr + bl + br)) / 2.5, 0.0, 1.0)

    display_gray_image(region)
    plt.clf();
    display_gray_image(norm_corners)
    plt.clf();
    return sqp, max(tup, tdp), circle

print("Corner detection on reds")
for i in range(len(red_roi)):
    roi = red_roi[i]
    (brect_x, brect_y, brect_w, brect_h), contour, red_ratio, blue_ratio, red_blue_ratio = roi
    sqp, trg, circle = corner_detection(roi, red_edges)
    red_roi[i] = ((brect_x, brect_y, brect_w, brect_h), contour,  red_ratio, blue_ratio, red_blue_ratio, sqp, trg, circle)

print("Corner detection on blues")
for i in range(len(blue_roi)):
    roi = blue_roi[i]
    (brect_x, brect_y, brect_w, brect_h), contour, red_ratio, blue_ratio, red_blue_ratio = roi
    sqp, trg, circle = corner_detection(roi, blue_edges)
    blue_roi[i] = ((brect_x, brect_y, brect_w, brect_h), contour, red_ratio, blue_ratio, red_blue_ratio, sqp, trg, circle)

## Geometrical Ratio Analysis

In [ ]:
# Mathematical ratios
circularity_ratios = {
    "circle": 1,
    "quadrilateral": 0.70,
    "octagon": 0.92,
    "triangle": 0.41,
    # "diamond": 0.64,
}

extent_ratios = {
    "circle": 0.785,
    "quadrilateral": 1,
    "octagon": 0.829,
    "triangle": 0.498,
    # "diamond": 0.5,
}

minextent_ratios = {
    "circle": 0.785,
    "quadrilateral": 1,
    "octagon": 0.829,
    "triangle": 0.498,
    # "diamond": 1
}

red_ratios = {
    "circle": 0.30,
    "quadrilateral": 0.0,
    "octagon": 0.65,
    "triangle": 0.20,
}

blue_ratios = {
    "circle": 0.60,
    "quadrilateral": 0.60,
    "octagon": 0.0,
    "triangle": 0.0,
}

output_classes = defaultdict(lambda: "unknown", {
    ("circle", "red"): "prohibitory",
    ("triangle", "red"): "priority",
    ("octagon", "red"): "stop",

    ("quadrilateral", "blue"): "information",
    ("circle", "blue"): "mandatory",
})

def detect_shape(roi, roi_type, return_probabilities = False):
    (brect_x, brect_y, brect_w, brect_h), contour, red_ratio, blue_ratio, red_blue_ratio, sqp, trg, circle = roi

    contourArea = cv.contourArea(contour)
    contourPerimeter = cv.arcLength(contour, True)
    # Bounding Rectangle Area - used to calculate extent of contour
    brect_area = brect_w * brect_h
    extent = contourArea / brect_area

    # Minimum Bounding Rectangle - takes into account orientation and fits the bounding rectangle thighly
    (min_brect_x0, min_brect_y0), (min_brect_x1, min_brect_y1), min_brect_angle = cv.minAreaRect(contour)
    min_brect_area = abs(min_brect_x0 - min_brect_x1) * abs(min_brect_y0 - min_brect_y1)

    min_extent = contourArea / min_brect_area

    # Circularity - measures how compact the contour is
    circularity = (4 * np.pi * contourArea) / (contourPerimeter * contourPerimeter + 1e-4)

    # Minimum Enclosing Circle - similar to circularity
    (min_circle_x, min_circle_y), circle_radius = cv.minEnclosingCircle(contour)
    min_circle_area = np.pi * circle_radius * circle_radius

    circle_extent = contourArea / min_circle_area

    metrics = ["circularity", "circle_extent", "extent", "min_extent"]
    ratios = [circularity, circle_extent, extent, min_extent]
    ratio_tables = [circularity_ratios, circularity_ratios, extent_ratios, minextent_ratios]
    if roi_type == "red":
        metrics.append("color_ratio")
        ratios.append(red_ratio)
        ratio_tables.append(red_ratios)
    elif roi_type == "blue":
        metrics.append("color_ratio")
        ratios.append(blue_ratio)
        ratio_tables.append(blue_ratios)
    
    metrics.append("corners")
    n_metrics = len(metrics)

    classes = ["circle", "quadrilateral", "octagon", "triangle"] # Add diamond
    n_classes = len(classes)

    probability_table = np.zeros(shape=(n_metrics + 1, n_classes + 1))
    for i in range(n_metrics - 1):
        ratio = ratios[i]
        table = ratio_tables[i]
        for j in range(n_classes):
            shape_class = classes[j]
            class_ratio = table[shape_class]
            probability_table[i, j] = abs(ratio - class_ratio) ** 2 # / (class_ratio + 1e-6)
        
        probability_table[i, n_classes] = np.sum(probability_table[i, 0:n_classes])
        probability_table[i, 0:n_classes] = (1 - (probability_table[i, 0:n_classes] / probability_table[i, n_classes])) / (n_classes - 1)
        probability_table[i, n_classes] = np.sum(probability_table[i, 0:n_classes])

    # Add corners row
    probability_table[n_metrics - 1, :n_classes] = np.array([circle, sqp, circle, trg])
    probability_table[n_metrics - 1, n_classes] = np.sum(probability_table[n_metrics - 1, 0:n_classes])
    probability_table[n_metrics - 1, :n_classes] /= probability_table[n_metrics - 1, n_classes]

    probability_table[-1, :n_classes] = np.mean(probability_table[:-1, :-1], axis=0)
    probability_table[-1, n_classes] = np.sum(probability_table[-1, 0:n_classes])

    chosen_shape = ""
    max_probability = -1
    for i, shape in enumerate(classes):
        p = probability_table[-1, i] 
        if p > max_probability:
            chosen_shape = shape
            max_probability = p

    CONFIDENCE = 0.1 # Confidence threshold, how much % is needed to obtain majority
    THRESHOLD = (1 / n_classes) * (1 + CONFIDENCE)
    if max_probability < THRESHOLD:
        chosen_shape = "other"
        max_probability = -1

    if roi_type == "red":
        if chosen_shape == "quadrilateral":
            chosen_shape = "other"
        elif chosen_shape == "circle" or chosen_shape == "octagon":
            if red_ratio > 0.45:
                chosen_shape = "octagon"
            else:
                chosen_shape = "circle"
    elif roi_type == "blue":
        if chosen_shape == "triangle" or chosen_shape == "octagon":
            chosen_shape = "other"

    if return_probabilities:
        df = pd.DataFrame(data=probability_table[:, :-1], columns=classes, index=metrics + ["AVG(P)"])
        return chosen_shape, max_probability, df, THRESHOLD

    return chosen_shape

output_image = src_image.copy()
print("Shape detection for red RoI")
for roi in red_roi:
    shape, prob, prob_table, threshold = detect_shape(roi, "red", return_probabilities=True)

    (x, y, w, h), contours, red_ratio, blue_ratio, red_blue_ratio, sqp, trg, circle = roi
    region = np.zeros_like(red_edges[y:y+h, x:x+w])
    cv.drawContours(region, [contours], 0, color=(255), offset=(-x, -y))

    print(f"Red Ratio: {red_ratio}")
    print(f"Blue Ratio: {blue_ratio}")
    print(f"Red Blue Ratio: {red_blue_ratio}")
    print(f"Detected {shape} with probability of {prob}. Minimum threshold was {threshold}")
    print(prob_table)
    fig, ax = display_gray_image(region)
    plt.clf();

    # Draw output
    if shape != "other":
        cv.drawContours(output_image, [contours], 0, color=(25, 255, 40), thickness=2)
        output_class = output_classes[(shape, "red")]
        cv.putText(output_image, output_class, (x, y - 5), cv.FONT_HERSHEY_SIMPLEX, 0.6, (25, 255, 40), 2, cv.LINE_AA)

print("Shape detection for blue RoI")
for roi in blue_roi:
    shape, prob, prob_table, threshold = detect_shape(roi, "blue", return_probabilities=True)

    (x, y, w, h), contours, red_ratio, blue_ratio, red_blue_ratio, sqp, trg, circle = roi
    region = np.zeros_like(blue_edges[y:y+h, x:x+w])
    cv.drawContours(region, [contours], 0, color=(255), offset=(-x, -y))

    print(f"Red Ratio: {red_ratio}")
    print(f"Blue Ratio: {blue_ratio}")
    print(f"Red Blue Ratio: {red_blue_ratio}")
    print(f"Detected {shape} with probability of {prob}. Minimum threshold was {threshold}")
    print(prob_table)
    fig, ax = display_gray_image(region)
    plt.clf();

    # Draw output
    if shape != "other":
        cv.drawContours(output_image, [contours], 0, color=(25, 255, 40), thickness=2)
        output_class = output_classes[(shape, "blue")]
        cv.putText(output_image, output_class, (x, y - 5), cv.FONT_HERSHEY_SIMPLEX, 0.6, (25, 255, 40), 2, cv.LINE_AA)

# Output

In [ ]:
display_bgr_image(output_image)